In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array , load_img
from tensorflow.keras.layers import Conv2D , Flatten , MaxPool2D , Dense , GlobalAveragePooling2D , AveragePooling2D , Dropout , Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
lr = 1e-4
epochs = 10
batch_size = 32

In [ ]:
directory = '/content/drive/My Drive/Colab files/face_mask_dataset_kaggle'
categories = ['with_mask' , 'without_mask']

In [ ]:
data=np.load("/content/drive/My Drive/Colab files/face_mask_dataset_kaggle/maskdataset.npz")


In [ ]:
data , labels = data['arr_0'] , data['arr_1']

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
data = np.array(data , dtype ='float32')
labels = np.array(labels)

In [ ]:
X_train , X_test, y_train , y_test = train_test_split(data , labels , test_size = 0.2 , random_state = 101)

In [ ]:
X_train.shape

(3066, 224, 224, 3)

In [ ]:
datagen = ImageDataGenerator(rotation_range=20,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest",
                            )

In [ ]:
base_model = MobileNetV2(weights='imagenet' , include_top= False , input_tensor=Input(shape = (224 , 224, 3)))

head_model = base_model.output
head_model= AveragePooling2D(7)(head_model)
head_model = Flatten()(head_model)
head_model = Dense( 128 , activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense( 2 , activation='softmax')(head_model)

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:

model = Model(inputs = base_model.input , outputs = head_model)

for layer in base_model.layers:
  layer.trainable = False

In [ ]:
optim = optimizers.Adam(learning_rate=lr , decay = lr/batch_size)
model.compile(optim , loss=tensorflow.keras.losses.binary_crossentropy , metrics=['accuracy'])

In [ ]:
model.fit(datagen.flow(X_train , y_train, batch_size = batch_size) , 
                  steps_per_epoch=len(X_train)//batch_size , 
                  validation_data=(X_test , y_test) , 
                  validation_steps= len(X_test)//batch_size , 
                  epochs = epochs)

Epoch 1/10
95/95 [==============================] - 30s 313ms/step - loss: 0.6744 - accuracy: 0.6684 - val_loss: 0.3764 - val_accuracy: 0.8566
Epoch 2/10
95/95 [==============================] - 29s 303ms/step - loss: 0.4601 - accuracy: 0.7914 - val_loss: 0.3183 - val_accuracy: 0.8866
Epoch 3/10
95/95 [==============================] - 29s 305ms/step - loss: 0.3925 - accuracy: 0.8312 - val_loss: 0.2863 - val_accuracy: 0.9035
Epoch 4/10
95/95 [==============================] - 29s 305ms/step - loss: 0.3711 - accuracy: 0.8454 - val_loss: 0.2796 - val_accuracy: 0.8957
Epoch 5/10
95/95 [==============================] - 29s 303ms/step - loss: 0.3303 - accuracy: 0.8668 - val_loss: 0.2413 - val_accuracy: 0.9126
Epoch 6/10
95/95 [==============================] - 29s 305ms/step - loss: 0.3203 - accuracy: 0.8662 - val_loss: 0.2271 - val_accuracy: 0.9166
Epoch 7/10
95/95 [==============================] - 29s 303ms/step - loss: 0.3205 - accuracy: 0.8593 - val_loss: 0.2318 - val_accuracy: 0.9140

In [ ]:
model_json = model.to_json()
with open("/content/drive/My Drive/Colab files/face_mask_dataset_kaggle/mask_detection.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Colab files/face_mask_dataset_kaggle/mask_detection.h5")
print("Saved model to disk")

Saved model to disk
